In [2]:
import joblib
data = joblib.load('nsmc.pki')

In [4]:
locals().update(data)

In [6]:
dtm

<2000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 12479 stored elements in Compressed Sparse Row format>

In [7]:
import pandas as pd
nsmc = pd.read_csv('../dataset/nsmc_train.csv', sep='\t')

In [12]:
x = dtm

In [15]:
y = nsmc.label.values[:2000]

In [13]:
from sklearn.model_selection import train_test_split

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.2, 
                                                    random_state=1984)

In [17]:
import tensorflow as tf

In [55]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [56]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
model.fit(x_train.A, y_train, epochs=3)

Epoch 1/3
50/50 [==============================] - 0s 2ms/step - loss: 0.6063 - accuracy: 0.8006
Epoch 2/3
50/50 [==============================] - 0s 2ms/step - loss: 0.5953 - accuracy: 0.8163
Epoch 3/3
50/50 [==============================] - 0s 2ms/step - loss: 0.5851 - accuracy: 0.8238


In [25]:
model.evaluate(x_test.A, y_test)

13/13 [==============================] - 0s 2ms/step - loss: 0.6174 - accuracy: 0.7275


[0.6173705458641052, 0.7275000214576721]

In [26]:
model.save('nsmc.krs')

INFO:tensorflow:Assets written to: nsmc.krs\assets


## 가중치 분석

In [28]:
model = tf.keras.models.load_model('nsmc.krs')

In [30]:
w, b = model.weights

In [33]:
word_sent = pd.DataFrame({'토큰': words, '가중치': w.numpy().flat})

In [37]:
word_sent.sort_values('가중치').head(10)

,토큰,가중치
537,아깝/VA,-0.253518
849,짜증/NNG,-0.252252
883,최악/NNG,-0.240776
331,뭐/NP,-0.240755
756,재미없/VA,-0.217700
533,쓰레기/NNG,-0.217405
856,찍/VV,-0.212211
593,없/VA,-0.201104
580,어설프/VA,-0.199846
140,내용/NNG,-0.196514


In [39]:
word_sent.sort_values('가중치').tail(10)

,토큰,가중치
304,명작/NNG,0.194084
298,멋지/VA,0.194863
291,매력/NNG,0.197248
111,나/NP,0.197513
417,사랑/NNG,0.200855
164,눈물/NNG,0.217433
757,재미있/VA,0.222085
852,짱/NNG,0.226019
881,최고/NNG,0.241623
759,재밌/VA,0.340268


## 희소행렬 변환

In [41]:
type(x_train)

scipy.sparse.csr.csr_matrix

In [42]:
# CSR -> COO 형식으로 바꿔줘야함.
x_coo = x_train.tocoo()

In [44]:
x_coo.row

array([   0,    0,    0, ..., 1599, 1599, 1599])

In [45]:
x_coo.col

array([  1, 119, 140, ..., 754, 951, 952])

In [46]:
x_coo.data

array([1, 1, 1, ..., 1, 1, 7], dtype=int64)

In [47]:
import numpy as np

In [48]:
index = np.column_stack((x_coo.row, x_coo.col))

In [49]:
index

array([[   0,    1],
       [   0,  119],
       [   0,  140],
       ...,
       [1599,  754],
       [1599,  951],
       [1599,  952]])

In [50]:
x_train_sparse = tf.SparseTensor(index, x_coo.data, x_coo.shape)

In [51]:
x_train_sparse = tf.sparse.reorder(x_train_sparse)

In [52]:
model.fit(x_train_sparse, y_train, epochs=1)

50/50 [==============================] - 0s 3ms/step - loss: 0.5752 - accuracy: 0.8281


## Early Stopping

In [57]:
model.fit(x_train.A, y_train, epochs=100, validation_split=0.1, 
         callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy')])

Epoch 1/100
45/45 [==============================] - 1s 6ms/step - loss: 0.6879 - accuracy: 0.5522 - val_loss: 0.6904 - val_accuracy: 0.5125
Epoch 2/100
45/45 [==============================] - 0s 3ms/step - loss: 0.6740 - accuracy: 0.6171 - val_loss: 0.6783 - val_accuracy: 0.5938
Epoch 3/100
45/45 [==============================] - 0s 3ms/step - loss: 0.6580 - accuracy: 0.7094 - val_loss: 0.6675 - val_accuracy: 0.6500
Epoch 4/100
45/45 [==============================] - 0s 4ms/step - loss: 0.6450 - accuracy: 0.7456 - val_loss: 0.6574 - val_accuracy: 0.7000
Epoch 5/100
45/45 [==============================] - 0s 4ms/step - loss: 0.6326 - accuracy: 0.7755 - val_loss: 0.6475 - val_accuracy: 0.7312
Epoch 6/100
45/45 [==============================] - 0s 4ms/step - loss: 0.6224 - accuracy: 0.7904 - val_loss: 0.6386 - val_accuracy: 0.7375
Epoch 7/100
45/45 [==============================] - 0s 4ms/step - loss: 0.6117 - accuracy: 0.8159 - val_loss: 0.6306 - val_accuracy: 0.7437
Epoch 8/100
4